# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

### Import data into database command
```sh
mongoimport --type json -d uk_food -c establishments --drop --jsonArray "Resources\establishments.json"
```

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'classDB',
 'config',
 'epa',
 'fruits_db',
 'local',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
db.establishments.find_one({})

{'_id': ObjectId('646523d9dac09d2506ef7643'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
insert_result.inserted_id

ObjectId('6465256582af4e1f3740aaab')

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [15]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
results = establishments.find_one({"BusinessTypeID": "Restaurant/Cafe/Canteen",
                                   "BusinessTypeID": {"$ne": ""}}, # our new restaurant does not have a value for this field
                                  ["BusinessTypeID", "BusinessType"])
results

{'_id': ObjectId('646523d9dac09d2506ef7643'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"},
                          [{"$set": {"BusinessTypeID": 1}}])

In [14]:
# Confirm that the new restaurant was updated
establishments.find_one({"BusinessName":"Penang Flavours"}, ["BusinessName", "BusinessTypeID"])

{'_id': ObjectId('6465256582af4e1f3740aaab'),
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [18]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
establishments.count_documents(dover_query)

994

In [19]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(dover_query)

In [20]:
# Check if any remaining documents include Dover
establishments.count_documents(dover_query)

0

In [21]:
# Check that other documents remain with 'find_one'
establishments.find_one({}, ["BusinessName", "LocalAuthorityName"])

{'_id': ObjectId('646523d9dac09d2506ef7924'),
 'BusinessName': 'The Pavilion',
 'LocalAuthorityName': 'Folkestone and Hythe'}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [26]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {"geocode.latitude": {'$toDouble': "$geocode.latitude"},
                                          "geocode.longitude": {'$toDouble': "$geocode.longitude"}
                                          } 
                                 }]
                           )

Use `update_many` to convert `RatingValue` to integer numbers.

In [23]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [24]:
# Change the data type from String to Integer for RatingValue
# toInt does not convert nulls
establishments.update_many({}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

In [29]:
# Check that the coordinates and rating value are now numbers
establishments.find_one({},
                        ["BusinessName", "geocode.latitude", "geocode.longitude", "RatingValue"])

{'_id': ObjectId('646523d9dac09d2506ef7924'),
 'BusinessName': 'The Pavilion',
 'RatingValue': 5,
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}